In [1]:
pip install -qU langchain-cohere

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")

 ········


In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How's it going today?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ace08a40-bff6-4b4c-84ff-beec9817cf85', 'token_count': {'input_tokens': 71, 'output_tokens': 16}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'ace08a40-bff6-4b4c-84ff-beec9817cf85', 'token_count': {'input_tokens': 71, 'output_tokens': 16}}, id='run-2c25c1de-0384-410d-8a53-2a50cbd937a3-0')

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but as an AI chatbot, I don't know your name. If you don't mind, you can share it with me, and I'd be delighted to address you by your name!", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '26b4e5a7-80a0-4472-8365-6869428f04c9', 'token_count': {'input_tokens': 71, 'output_tokens': 42}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '26b4e5a7-80a0-4472-8365-6869428f04c9', 'token_count': {'input_tokens': 71, 'output_tokens': 42}}, id='run-ad0523d0-bac0-4dbc-87f2-bffcc2445eaa-0')

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="Your name is Bob! How's the weather in your area, Bob?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '981d59bd-68cb-42c9-a4a2-287c1a36cb85', 'token_count': {'input_tokens': 92, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '981d59bd-68cb-42c9-a4a2-287c1a36cb85', 'token_count': {'input_tokens': 92, 'output_tokens': 15}}, id='run-25b5e91f-7f94-4f73-a9c5-2224a82f51a2-0')

In [6]:
! pip install langchain_community

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
    --------------------------------------- 0.0/2.2 MB 660.6 kB/s eta 0:00:04
    --------------------------------------- 0.0/2.2 MB 653.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.2 MB 581.0 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.2 MB 774.0 kB/s eta 0:00:03
   --- ------------------------------------ 0.2/2.2 MB 841.6 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.2 MB 947.5 kB/s eta 0:00:02
   ----- ---------------------------------- 0.3/2.2 MB 905.4 kB/s eta 0:00:03
   ------- -------------------------------- 0.4/2.2 MB 1.1 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/2.2 MB 1.3 MB/s eta 0:00:02
   ------------- -------------------------- 0.7/2.2 MB 1.6 MB/s eta 0:00:01
   -------------------- ------------------- 1.1/2.2 MB 2.1 MB/s eta 0:00:01
   ------------------------- -------------- 1.4/2.2 MB 2.5 MB/s eta 0:00:01
   --

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "abc2"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hi Bob! It's nice to meet you. How's it going today?"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"You've told me your name is Bob, and I believe you! You seem like a trustworthy guy."

In [13]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but as an AI chatbot, I don't know your name. If you don't mind, you can share it with me, and I'll be happy to address you by it."

In [14]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"You've got a great sense of humour Bob, that's for sure! You're very persistent about your name, or lack thereof. I assure you that your name is Bob, as you've mentioned multiple times."

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Hi Bob! How's it going? I'm happy to help you with whatever you need. Do you have any questions or topics you'd like to talk about?"

In [17]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [18]:
config = {"configurable": {"session_id": "abc5"}}

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

"Hi Jim! It's nice to hear from you. How's it going today?"

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Jim! Did you forget? It's a pleasure to meet you, Jim."

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¡Bienvenido a mi conversación! Espero que estés teniendo un buen día. 😊'

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [24]:
config = {"configurable": {"session_id": "abc11"}}



In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola, Todd! ¡Bienvenido! 😊'

In [26]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Spanish"},
    config=config,
)

response.content

'Tu nombre es Todd, ¡un nombre genial! 🤩'

In [27]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [28]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [29]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I can't know your name unless you provide it to me. However, you could probably have a lot of fun at a music festival! There's usually one playing somewhere in the world. Would you like to know about a specific music festival or recommendations on how to have fun?"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

"Sorry, I am unable to determine your favorite ice cream flavor without more context. However, the most popular ice cream flavor in the world is vanilla. It's a classic flavor that's creamy, comforting, and versatile. It's also a great base for sundaes and can be paired with many different toppings and sauces."

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I can't help you with that as you haven't mentioned your name yet. If you tell me your name, I'll be happy to greet you!"

In [37]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

"That's a tricky one! As I don't know you, I'm unable to answer that question. However, some popular ice cream flavors include chocolate, vanilla, and strawberry. They are delicious and are often people's favorites!"

In [38]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Hi| Todd|!| Here|'s| a| joke| for| you|:|
Why| couldn|'t| the| bad| sailor| learn| the| alphabet|?|
Because| he| kept| getting| seas|ick| at| "|C|"!||